# Anàlisi del text

Intro

In [1]:
import pandas as pd 
filename = 'tocho2.csv'
df = pd.read_csv(filename, index_col = 0)
df['text_ tokens'] = df['text_ tokens'].str.split('\t')
print(df['text_ tokens'].iloc[0])

['101', '16493', '12478', '117', '10105', '42370', '76299', '100', '187', '15480', '10108', '23837', '117', '10393', '169', '32342', '18077', '10135', '10226', '27925', '10111', '10393', '17087', '10950', '10114', '41904', '51937', '16981', '10142', '15217', '119', '14820', '10689', '17860', '169', '28502', '11360', '10689', '13461', '10105', '13451', '35688', '26981', '10142', '12557', '136', '14120', '131', '120', '120', '188', '119', '11170', '120', '175', '11166', '15705', '10129', '19282', '10113', '11396', '10362', '102']


Volem identificar l'idioma dels twits, codificats en [BERT Base Multilingual Cased](https://huggingface.co/bert-base-multilingual-cased). Com els valor de la columna language estan codificats, volem inspeccionar el contingut dels textos per trobar els twits escrits en llengües que poguem interpretar. 

Per això decodifiquem el text d'un twit per valor de `language` i llegim el seu contingut. Farem servir la lliberia *transformers*:

In [2]:
#Import of the BertTokenizer module to detokenize the text tokens
from transformers import BertTokenizer

#Setting the BERT Base Multilingual Cased tokenizer model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

#Print the first tweet in the most frequent languages
for index in df.language.value_counts().index[:21]:
    dl = df[df.language == index].copy()
    dl = dl.reset_index(drop=True)
    print("language = {} \n".format(index))        
    print(tokenizer.decode(dl['text_ tokens'].iloc[0]) + '\n')

Neither PyTorch nor TensorFlow >= 2.0 have been found.Models won't be available and only tokenizers, configurationand file/data utilities can be used.


language = D3164C7FBCF2565DDF915B1B3AEFB1DC 

[CLS] Jacob Young, the Syndicate [UNK] s Director of PR, has a serious problem on his hands and has reached out to Hammond Robotics for help. Can they find a solution before they become the next ones marked for death? https : / / t. co / g5evrQa8va [SEP]

language = 22C448FF81263D4BAF2A176145EE9EAD 

[CLS] ただいま うさ 顔 洗 い 中. おしっこ 飛 ばしの 後 にはいつもする 仕 草. 壁 跡 は 生 々しいけれど 、 なんとも 愛 らしい [UNK] [UNK] ！.. # うさぎ # うさぎ 好 きさんと 繋 がりたい # 顔 洗 い # キリトリセカイ # 癒 し 動 画 # 動 物 好 きな 人 と 繋 がりたい # うさぎあるある # 動 物 あるある # ファインダー 越 しの 私 の 世 界 # 部 屋 んぽ # かわいい https : / / t. co / 2gKb0EeWqa [SEP]

language = 06D61DCBBE938971E1EA0C38BD9B5446 

[CLS] RT @ GermantequilIa : otra vez Las Palmas hacia ruta 68 pista izquierda bloqueada por choque de vehículo @ sitiodelsuceso @ AlertaVR https : / / t. [UNK] [SEP]

language = ECED8A16BE2A5E8871FD55F4842F16B1 

[CLS] Porque ele matou, esquartejou e deu os restos mortais da mãe do filho dele para os cachorros só pq não queria pagar pensã

De cara a l'anàlisi textual dels twits, només hem inclòs aquells en **anglès**, **castellà** i **català** filtrant mitjançant els codis corresponents als textos que hem identificat.

Per tant, només farem servir els languages `D3164C7FBCF2565DDF915B1B3AEFB1DC` (anglès), `06D61DCBBE938971E1EA0C38BD9B5446` (castellà) i `190BA7DA361BC06BC1D7E824C378064D` (català).

In [3]:
#Subset: English language 
language = 'D3164C7FBCF2565DDF915B1B3AEFB1DC'
d_eng = df[df.language == language].copy()
d_eng = d_eng.reset_index(drop=True)

#Subset: Spanish language 
language = '06D61DCBBE938971E1EA0C38BD9B5446'
d_cast = df[df.language == language].copy()
d_cast = d_cast.reset_index(drop=True)

#Subset: Catalan language 
language = '190BA7DA361BC06BC1D7E824C378064D'
d_cat = df[df.language == language].copy()
d_cat = d_cat.reset_index(drop=True)

print("English subset has a total of {} twits".format(d_eng.size))
print("Spanish subset has a total of {} twits".format(d_cast.size))
print("Catalan subset has a total of {} twits".format(d_cat.size))

English subset has a total of 4579224 twits
Spanish subset has a total of 999888 twits
Catalan subset has a total of 25272 twits


Una vegada filtrats els twits per idioma, parem atenció als tipus de twit:

In [4]:
d_eng['tweet_type'].value_counts()

TopLevel    99974
Retweet     72282
Quote       18545
Name: tweet_type, dtype: int64

Tenim tres tipus de twits:
- <b>TopLevel</b>: Twit publicat originalment per un usuari
- <b>Retweet</b>: Twit compartit per un usuari, però publicat originalment per un altre usuari
- <b>Quote</b>: Resposta d'un usuari a un TopLevel ó Retweet

Amb la intenció d'identificar contingut original per usuari, descartem els Retweets. 

In [5]:
engagement_user_type = 'engaged_with_user_id'
tweet_type = 'TopLevel'
d_eng_top_level = d_eng.loc[d_eng['tweet_type'] == tweet_type]

Així doncs, podem obtenir els usuaris amb més publicacions originals (TopLevel) de cada idioma.

In [6]:
d_eng_top_level[engagement_user_type].value_counts()[:10]

C6758D692A850E4C67B2763B66D1CFA8    659
5FF622786FB4924A067BD44D4B717570    509
E5D1B83B0E02FAFF871EEEF276D18132    395
7C03844E8B2E0C7B4346D41028AB14E2    328
FBB188A3C1E05C41587AAAC00B5B1879    232
F2A8BF0F4EB185E6D2E5E1A0DF4C33AE    204
416B919C0DAA48D42FF6780574034149    189
8A800256378089EF53C6F655F8690490    183
A9DAB08351D94BDE86235B37D6E8C61D    182
9D9C2BC354011249F2D4D9B9C4205AC9    168
Name: engaged_with_user_id, dtype: int64

Pel primer anàlisi, farem servir l'usuari més 'actiu' en anglès. 

In [7]:
#Take the user ID with most publications
engaging_user_id = d_eng_top_level[engagement_user_type].value_counts().index[0]

#Subset with only the twits of the user ID with most publications
d_eng_user = d_eng_top_level.loc[d_eng[engagement_user_type] == engaging_user_id]

In [8]:
for n in range (10): print(tokenizer.decode(d_eng_user['text_ tokens'].iloc[n]))

[CLS] Why didn [UNK] t John Bolton complain about this [UNK] nonsense [UNK] a long time ago, when he was very publicly terminated. He said, not that it matters, NOTHING! [SEP]
[CLS] [UNK] I like President Trump [UNK] s Tweets ( Social Media ), I like everything about him... and this Ukraine stuff, the trial, the impeachment, this isn [UNK] t t about Ukraine. Donald Trump has committed the two unpardonable sins in the eyes of the Democrats. He beat Hillary Clinton in 2016, and.. [SEP]
[CLS] Are you better off now than you were three years ago? Almost everyone say YES! [SEP]
[CLS] Reports are that basketball great Kobe Bryant and three others have been killed in a helicopter crash in California. That is terrible news! [SEP]
[CLS] The Democrat controlled House never even asked John Bolton to testify. It is up to them, not up to the Senate! [SEP]
[CLS]..... So, what the hell has happened to @ FoxNews. Only I know! Chris Wallace and others should be on Fake News CNN or MSDNC. How [UNK] s Sh

## Text tokens: anàlisi i data cleaning

Imprimim els primers tokens del model per veure si es segueix algun ordre en la descripció de tokens. 

In [9]:
n = 1000
print(tokenizer.decode(x for x in range(0,n)))

[PAD] [unused1] [unused2] [unused3] [unused4] [unused5] [unused6] [unused7] [unused8] [unused9] [unused10] [unused11] [unused12] [unused13] [unused14] [unused15] [unused16] [unused17] [unused18] [unused19] [unused20] [unused21] [unused22] [unused23] [unused24] [unused25] [unused26] [unused27] [unused28] [unused29] [unused30] [unused31] [unused32] [unused33] [unused34] [unused35] [unused36] [unused37] [unused38] [unused39] [unused40] [unused41] [unused42] [unused43] [unused44] [unused45] [unused46] [unused47] [unused48] [unused49] [unused50] [unused51] [unused52] [unused53] [unused54] [unused55] [unused56] [unused57] [unused58] [unused59] [unused60] [unused61] [unused62] [unused63] [unused64] [unused65] [unused66] [unused67] [unused68] [unused69] [unused70] [unused71] [unused72] [unused73] [unused74] [unused75] [unused76] [unused77] [unused78] [unused79] [unused80] [unused81] [unused82] [unused83] [unused84] [unused85] [unused86] [unused87] [unused88] [unused89] [unused90] [unused91] [u

Veiem que els primers 105 tokens estan reservats per codis . Després s'assignen els tokens posteriors a  signes de puntuació i números. Desprès es codifiquen els distints alfabets que s'inclouen al model. 

Per un altre costat, analiztant tokens més endavant, podem veure que si a cada paraula reconeguda pel model, se li assignarà un token propi.

In [10]:
print(tokenizer.decode(x for x in range(11000,11200)))

33 годинеן three 1948 fuů invånaream kvadratkilometerou4 Earthä anche benот 1942는 made англria కి yeov 00 1957 người 1930 1920il used 1954mo dhe 09 vomwa Unjo sebagai formaésicabe Seion durante przez suchного deux 08 known ved 36 South ta nie then он USAken 1946 dass You Mariaina να 34 final 1955 से である Leagueной کی populationпраland का 1938lla than nuب 1953ine naamTسный 300 IV 1949 Thomas 1918 India τον 1952 All한 הוא State merupakanскойunні Oxford familie club July 1947 یکzione més elle Richardche June team Az 1936 unter học 44Pরneric care secondł zen ca mga Nel として Retrievedok відigth Marchو inomnuш She που には someada Santa there olehste GeoNamesut British pe Film ke West data5 gli areaán co 38 һәм 1937ció born 400 kao any became τωνdos vớitonского serie एक ook છેque vorку place Berlinныеty soorttes includingে


### Text cleaning

Podem veure al text que es repeteixen uns codis a l'inici (`[CLS]`) i final (`[SEP]`) de cada twit. De cara al procès de data cleaning, ens interessaria eliminar aquests codis. Aprofitarem també per eliminar qualsevol possible aparició d'altres tokens que no aportaran valor al text: `[PAD]` i `[MASK]`. 

In [11]:
num_tokens = [0, 101, 102, 103]

for x in num_tokens:
    print('{}: {}'.format(x, tokenizer.decode(x)))

0: [ P A D ]
101: [ C L S ]
102: [ S E P ]
103: [ M A S K ]


Una vegada identificat el token per cadascun d'aquets codis, procedim a eliminar-los del nostre dataset.

In [12]:
#Convert the token list to string items
num_tokens = [ str(x) for x in num_tokens ]

#Remove all the items in num_tokens in each tweet
for k in range(len(d_eng['text_ tokens'])):
    for r in num_tokens:
        if r in d_eng['text_ tokens'][k]:
            d_eng['text_ tokens'][k].remove(r)

Podem veure la diferència imprimint els mateixos twits de l'usuari més actiu.

In [13]:
d_eng_top_level = d_eng.loc[d_eng['tweet_type'] == tweet_type]
engaging_user_id = d_eng_top_level[engagement_user_type].value_counts().index[0]
d_eng_user = d_eng_top_level.loc[d_eng[engagement_user_type] == engaging_user_id]

for n in range (10): print(tokenizer.decode(d_eng_user['text_ tokens'].iloc[n]))

Why didn [UNK] t John Bolton complain about this [UNK] nonsense [UNK] a long time ago, when he was very publicly terminated. He said, not that it matters, NOTHING!
[UNK] I like President Trump [UNK] s Tweets ( Social Media ), I like everything about him... and this Ukraine stuff, the trial, the impeachment, this isn [UNK] t t about Ukraine. Donald Trump has committed the two unpardonable sins in the eyes of the Democrats. He beat Hillary Clinton in 2016, and..
Are you better off now than you were three years ago? Almost everyone say YES!
Reports are that basketball great Kobe Bryant and three others have been killed in a helicopter crash in California. That is terrible news!
The Democrat controlled House never even asked John Bolton to testify. It is up to them, not up to the Senate!
..... So, what the hell has happened to @ FoxNews. Only I know! Chris Wallace and others should be on Fake News CNN or MSDNC. How [UNK] s Shep Smith doing? Watch, this will be the beginning of the end for 

## Links: enllaços, multimedia i retweets

In [14]:
import re
#We are looking for patterns such as https : / / t. co / yvMa6bPqfy
valid_pattern = 'https : \/ \/ t\. co \/ [\dA-Za-z\.-]+'

In [15]:
list_twits = []

#Gathering all the strings that meet the pattern
for index, row in d_eng_user.iterrows(): 
    list_twits.append(re.findall(valid_pattern, 
                                 tokenizer.decode(d_eng['text_ tokens'][index])))
    
    #print(tokenizer.decode(d_tweet_type['text_ tokens'][index]))

In [16]:
#Let's print them in order to check the resulting strings and easily access the links in the text
for i, s in enumerate(list_twits):
    list_twits_item = []
    for item in list_twits[i]:
        link_regex = item.replace(' ','')
        list_twits_item.append(link_regex) # I use replace because s.strip() is not working 
        print(link_regex) #Let's print the link so we can access it easier
    
    #list_twits_fix.append(list_twits_item)
    if list_twits_item != []: print(list_twits_item) #Print only the lists that are not empty
#print(list_twits_fix)

https://t.co/drhi6UeJ14
['https://t.co/drhi6UeJ14']
https://t.co/yvMa6bPqfy
['https://t.co/yvMa6bPqfy']
https://t.co/drhi6UeJ14
['https://t.co/drhi6UeJ14']
https://t.co/4YCo01XYCn
['https://t.co/4YCo01XYCn']
https://t.co/UCdQXY3vPn
['https://t.co/UCdQXY3vPn']
https://t.co/QwYzS32lyQ
['https://t.co/QwYzS32lyQ']
https://t.co/UCdQXY3vPn
['https://t.co/UCdQXY3vPn']
https://t.co/WFK33pR0Lv
['https://t.co/WFK33pR0Lv']
https://t.co/SOn6wRV9Zs
['https://t.co/SOn6wRV9Zs']
https://t.co/VshQceiwUA
['https://t.co/VshQceiwUA']
https://t.co/5NeC0mFWfU
['https://t.co/5NeC0mFWfU']
https://t.co/rpGHpdoIa6
['https://t.co/rpGHpdoIa6']
https://t.co/9LXDf6mJKf
['https://t.co/9LXDf6mJKf']
https://t.co/rrtF1Stk78
['https://t.co/rrtF1Stk78']
https://t.co/UCdQXY3vPn
['https://t.co/UCdQXY3vPn']
https://t.co/QwYzS32lyQ
['https://t.co/QwYzS32lyQ']
https://t.co/rrtF1Stk78
['https://t.co/rrtF1Stk78']
https://t.co/VshQceiwUA
['https://t.co/VshQceiwUA']
https://t.co/yvMa6bPqfy
['https://t.co/yvMa6bPqfy']
https://t.co

In [17]:
#Create a new column to store the list of links
d_eng['present_links_url'] = ""
#d_eng.at[index, 'present_links_text'] = links_list

for index, row in d_eng.iterrows():
    links_list = []
    links_list = re.findall(valid_pattern, tokenizer.decode(d_eng['text_ tokens'][index]))
    for i, s in enumerate(links_list):
        links_list[i] = s.replace(' ','') # I use replace because s.strip() is not working 
        
    d_eng.at[index, 'present_links_url'] = links_list

Per tal de agrupar totes les característiques textuals, reorganitzem les columnes del dataset.

In [18]:
col = d_eng['present_links_url']
d_eng.drop(labels=['present_links_url'], axis=1, inplace = True)
d_eng.insert(1, 'present_links_url', col)
d_eng.head()

,text_ tokens,present_links_url,hashtags,tweet_id,present_media,present_links,present_domains,tweet_type,language,tweet_timestamp,...,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engaging_user_account_creation,engagee_follows_engager,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp
0,"[16493, 12478, 117, 10105, 42370, 76299, 100, ...",[https://t.co/g5evrQa8va],NaN,39024FBE0136E046D1357196BAECFCA6,GIF,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581532200,...,000006829BEADA9EEA695CF0C334B426,2,29,False,1568107028,False,NaN,NaN,NaN,NaN
1,"[14924, 16118, 10114, 12888, 15127, 31204, 101...",[https://t.co/PxG5XzTruM],3653868A966576CF17D6A9064889BCED\t7A6710E791A1...,AB3ADBBD011F88D10FE7F6C5FDAB214C,Photo,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581087590,...,000269A50293CC2B1E9C72B287FA624E,515,424,False,1254588643,True,NaN,NaN,NaN,1.581090e+09
2,"[138, 19826, 10108, 10105, 10635, 10105, 12250...",[],NaN,675D7920EA2FB4869BA767F5122FB115,NaN,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581134845,...,0004C10C480EF88F3EED5822149DC93F,4,125,False,1547955941,False,NaN,NaN,NaN,1.581135e+09
3,"[56898, 137, 42374, 36630, 10797, 90861, 131, ...",[],NaN,7EAAE5C243D675AD4CFC2C052B0E0BB5,NaN,NaN,NaN,Retweet,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581011517,...,000830B836532B8FC5564949E94FEDD6,33439,35598,False,1261862258,True,NaN,1.581012e+09,NaN,NaN
4,"[137, 87043, 10679, 11369, 11211, 22650, 11305...",[],C8F2526CD1AEA3A8CFC67EB4B79E46D5,E243DF6401359C38C5D21B6FB86A881E,NaN,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581037782,...,0011F45B80474E4FF47250ED24C5482C,27980,25991,False,1493066269,True,NaN,NaN,NaN,1.581058e+09


In [19]:
#Replace the links in the text in order to made them clickable

#Now that the text will be more readable, we will create a column with the text
d_eng['text'] = ""    

for index, row in d_eng.iterrows():
    links_list = []
    tweet_text = tokenizer.decode(d_eng['text_ tokens'][index])
    links_list = re.findall(valid_pattern, tweet_text)
    
    for i, s in enumerate(links_list):
        if i != 0:
            tweet_text = tweet_text_replaced
        link_old = s
        link_new = s.replace(' ','') # I use replace because s.strip() is not working
        tweet_text_replaced = tweet_text.replace(link_old, link_new)
    
    if links_list == []:    
        d_eng.at[index, 'text'] = tweet_text
    else:
        d_eng.at[index, 'text'] = tweet_text_replaced

Per tal de agrupar totes les característiques textuals, reorganitzem les columnes del dataset.

In [20]:
second_col = d_eng['text']
d_eng.drop(labels=['text'], axis=1, inplace = True)
d_eng.insert(1, 'text', second_col)
d_eng.head()

,text_ tokens,text,present_links_url,hashtags,tweet_id,present_media,present_links,present_domains,tweet_type,language,...,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engaging_user_account_creation,engagee_follows_engager,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp
0,"[16493, 12478, 117, 10105, 42370, 76299, 100, ...","Jacob Young, the Syndicate [UNK] s Director of...",[https://t.co/g5evrQa8va],NaN,39024FBE0136E046D1357196BAECFCA6,GIF,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,...,000006829BEADA9EEA695CF0C334B426,2,29,False,1568107028,False,NaN,NaN,NaN,NaN
1,"[14924, 16118, 10114, 12888, 15127, 31204, 101...",Final week to see my exhibition in the Shircli...,[https://t.co/PxG5XzTruM],3653868A966576CF17D6A9064889BCED\t7A6710E791A1...,AB3ADBBD011F88D10FE7F6C5FDAB214C,Photo,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,...,000269A50293CC2B1E9C72B287FA624E,515,424,False,1254588643,True,NaN,NaN,NaN,1.581090e+09
2,"[138, 19826, 10108, 10105, 10635, 10105, 12250...",A lot of the time the side that wins is simply...,[],NaN,675D7920EA2FB4869BA767F5122FB115,NaN,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,...,0004C10C480EF88F3EED5822149DC93F,4,125,False,1547955941,False,NaN,NaN,NaN,1.581135e+09
3,"[56898, 137, 42374, 36630, 10797, 90861, 131, ...","RT @ readyletsgo27 : Schiff, Nadler and Pelosi...",[],NaN,7EAAE5C243D675AD4CFC2C052B0E0BB5,NaN,NaN,NaN,Retweet,D3164C7FBCF2565DDF915B1B3AEFB1DC,...,000830B836532B8FC5564949E94FEDD6,33439,35598,False,1261862258,True,NaN,1.581012e+09,NaN,NaN
4,"[137, 87043, 10679, 11369, 11211, 22650, 11305...",@ LiliyaL62076921 Unique beauty of flowers [UN...,[],C8F2526CD1AEA3A8CFC67EB4B79E46D5,E243DF6401359C38C5D21B6FB86A881E,NaN,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,...,0011F45B80474E4FF47250ED24C5482C,27980,25991,False,1493066269,True,NaN,NaN,NaN,1.581058e+09


In [21]:
#We'll use a list of 10 random ints in order to print 10 random tweets 
import random
randomlist = []
for i in range(0,19):
    n = random.randint(0,100)
    randomlist.append(n)

In [22]:
d_eng_top_level = d_eng.loc[d_eng['tweet_type'] == tweet_type]
engaging_user_id = d_eng_top_level[engagement_user_type].value_counts().index[0]
d_eng_user = d_eng_top_level.loc[d_eng[engagement_user_type] == engaging_user_id]

for n in randomlist: 
    links_list = d_eng_user['present_links_url'].iloc[n]
#    print(d_eng_user['engaged_with_user_id'].iloc[n] + ': \n\n'  + 
#                           'Original text: \n' + tokenizer.decode(d_eng_user['text_ tokens'].iloc[n]) + '\n\n' +
#                           'Resulting text: \n' + d_eng_user['text'].iloc[n] + '\n\n' +
#                           'Links: ' + '{} \n'.format(d_eng_user['present_links_url'].iloc[n]) + '\n\n' +
#                           '---------------' + '\n\n' )
    print('User ID: ' + d_eng_user['engaged_with_user_id'].iloc[n] + '\n'  +
          'Tweet ID: ' + d_eng_user['tweet_id'].iloc[n] + '\n\n'  +
          'Original text: \n' + tokenizer.decode(d_eng_user['text_ tokens'].iloc[n]) + '\n\n' +
          'Resulting text: \n' + d_eng_user['text'].iloc[n] + '\n')
    if links_list != []:
        print ('Links: ')
        for item in links_list:
            print(item)
    print('------------------------ \n')

User ID: C6758D692A850E4C67B2763B66D1CFA8
Tweet ID: 8794A7179ACFDE83C69FFE47E27D932F

Original text: 
Really pathetic how @ FoxNews is trying to be so politically correct by loading the airwaves with Democrats like Chris Van Hollen, the no name Senator from Maryland. He has been on forever playing up the Impeachment Hoax. Dems wouldn [UNK] t even give Fox their low ratings debates....

Resulting text: 
Really pathetic how @ FoxNews is trying to be so politically correct by loading the airwaves with Democrats like Chris Van Hollen, the no name Senator from Maryland. He has been on forever playing up the Impeachment Hoax. Dems wouldn [UNK] t even give Fox their low ratings debates....

------------------------ 

User ID: C6758D692A850E4C67B2763B66D1CFA8
Tweet ID: 22B556836CFEA0B0AD0C231079188AE0

Original text: 
BIGGEST TRADE DEAL EVER MADE, the USMCA, was signed yesterday and the Fake News Media barely mentioned it. They never thought it could be done. They have zero credibility!

Resul

In [23]:
#Create text with no links
d_eng['text_tokens_no_links'] = ""    

for index, row in d_eng.iterrows():
    links_list = []
    tweet_text = tokenizer.decode(d_eng['text_ tokens'][index])
    links_list = re.findall(valid_pattern, tweet_text)
    
    for i, s in enumerate(links_list):
        if i != 0:
            tweet_text = tweet_text_replaced
        link_old = s
        tweet_text_replaced = tweet_text.replace(link_old, '')
    
    if links_list == []:
        d_eng.at[index, 'text_tokens_no_links'] = d_eng['text_ tokens'][index]        
    else:
        new_tokens = tokenizer.encode(tweet_text_replaced)
        new_tokens = [str(x) for x in new_tokens]
        d_eng.at[index, 'text_tokens_no_links'] = new_tokens

for k in range(len(d_eng['text_tokens_no_links'])):
    for r in num_tokens:
        if r in d_eng['text_tokens_no_links'][k]:
            d_eng['text_tokens_no_links'][k].remove(r)

In [24]:
second_col = d_eng['text_tokens_no_links']
d_eng.drop(labels=['text_tokens_no_links'], axis=1, inplace = True)
d_eng.insert(1, 'text_tokens_no_links', second_col)            
            
d_eng.head()

,text_ tokens,text_tokens_no_links,text,present_links_url,hashtags,tweet_id,present_media,present_links,present_domains,tweet_type,...,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engaging_user_account_creation,engagee_follows_engager,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp
0,"[16493, 12478, 117, 10105, 42370, 76299, 100, ...","[16493, 12478, 117, 10105, 42370, 76299, 100, ...","Jacob Young, the Syndicate [UNK] s Director of...",[https://t.co/g5evrQa8va],NaN,39024FBE0136E046D1357196BAECFCA6,GIF,NaN,NaN,TopLevel,...,000006829BEADA9EEA695CF0C334B426,2,29,False,1568107028,False,NaN,NaN,NaN,NaN
1,"[14924, 16118, 10114, 12888, 15127, 31204, 101...","[14924, 16118, 10114, 12888, 15127, 31204, 101...",Final week to see my exhibition in the Shircli...,[https://t.co/PxG5XzTruM],3653868A966576CF17D6A9064889BCED\t7A6710E791A1...,AB3ADBBD011F88D10FE7F6C5FDAB214C,Photo,NaN,NaN,TopLevel,...,000269A50293CC2B1E9C72B287FA624E,515,424,False,1254588643,True,NaN,NaN,NaN,1.581090e+09
2,"[138, 19826, 10108, 10105, 10635, 10105, 12250...","[138, 19826, 10108, 10105, 10635, 10105, 12250...",A lot of the time the side that wins is simply...,[],NaN,675D7920EA2FB4869BA767F5122FB115,NaN,NaN,NaN,TopLevel,...,0004C10C480EF88F3EED5822149DC93F,4,125,False,1547955941,False,NaN,NaN,NaN,1.581135e+09
3,"[56898, 137, 42374, 36630, 10797, 90861, 131, ...","[56898, 137, 42374, 36630, 10797, 90861, 131, ...","RT @ readyletsgo27 : Schiff, Nadler and Pelosi...",[],NaN,7EAAE5C243D675AD4CFC2C052B0E0BB5,NaN,NaN,NaN,Retweet,...,000830B836532B8FC5564949E94FEDD6,33439,35598,False,1261862258,True,NaN,1.581012e+09,NaN,NaN
4,"[137, 87043, 10679, 11369, 11211, 22650, 11305...","[137, 87043, 10679, 11369, 11211, 22650, 11305...",@ LiliyaL62076921 Unique beauty of flowers [UN...,[],C8F2526CD1AEA3A8CFC67EB4B79E46D5,E243DF6401359C38C5D21B6FB86A881E,NaN,NaN,NaN,TopLevel,...,0011F45B80474E4FF47250ED24C5482C,27980,25991,False,1493066269,True,NaN,NaN,NaN,1.581058e+09


In [25]:
#Inspect different types of tweets in order to understand which kind of links we have

In [26]:
#Extreure hashtags

In [27]:
#Replace the hashtags in the text 
valid_pattern = '# [a-z0-9]+'

#Create a new column to store the list of links
d_eng['hashtags_text'] = ""

for index, row in d_eng.iterrows():
    links_list = []
    links_list = re.findall(valid_pattern, tokenizer.decode(d_eng['text_ tokens'][index]))
    for i, s in enumerate(links_list):
        links_list[i] = s.replace(' ','') # I use replace because s.strip() is not working 
        
    d_eng.at[index, 'hashtags_text'] = links_list

In [28]:
col = d_eng['hashtags_text']
d_eng.drop(labels=['hashtags_text'], axis=1, inplace = True)
d_eng.insert(3, 'hashtags_text', col)            
            
d_eng.head()

,text_ tokens,text_tokens_no_links,text,hashtags_text,present_links_url,hashtags,tweet_id,present_media,present_links,present_domains,...,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engaging_user_account_creation,engagee_follows_engager,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp
0,"[16493, 12478, 117, 10105, 42370, 76299, 100, ...","[16493, 12478, 117, 10105, 42370, 76299, 100, ...","Jacob Young, the Syndicate [UNK] s Director of...",[],[https://t.co/g5evrQa8va],NaN,39024FBE0136E046D1357196BAECFCA6,GIF,NaN,NaN,...,000006829BEADA9EEA695CF0C334B426,2,29,False,1568107028,False,NaN,NaN,NaN,NaN
1,"[14924, 16118, 10114, 12888, 15127, 31204, 101...","[14924, 16118, 10114, 12888, 15127, 31204, 101...",Final week to see my exhibition in the Shircli...,"[#vincennesuniversity, #shircliffgallery, #ins...",[https://t.co/PxG5XzTruM],3653868A966576CF17D6A9064889BCED\t7A6710E791A1...,AB3ADBBD011F88D10FE7F6C5FDAB214C,Photo,NaN,NaN,...,000269A50293CC2B1E9C72B287FA624E,515,424,False,1254588643,True,NaN,NaN,NaN,1.581090e+09
2,"[138, 19826, 10108, 10105, 10635, 10105, 12250...","[138, 19826, 10108, 10105, 10635, 10105, 12250...",A lot of the time the side that wins is simply...,[],[],NaN,675D7920EA2FB4869BA767F5122FB115,NaN,NaN,NaN,...,0004C10C480EF88F3EED5822149DC93F,4,125,False,1547955941,False,NaN,NaN,NaN,1.581135e+09
3,"[56898, 137, 42374, 36630, 10797, 90861, 131, ...","[56898, 137, 42374, 36630, 10797, 90861, 131, ...","RT @ readyletsgo27 : Schiff, Nadler and Pelosi...",[],[],NaN,7EAAE5C243D675AD4CFC2C052B0E0BB5,NaN,NaN,NaN,...,000830B836532B8FC5564949E94FEDD6,33439,35598,False,1261862258,True,NaN,1.581012e+09,NaN,NaN
4,"[137, 87043, 10679, 11369, 11211, 22650, 11305...","[137, 87043, 10679, 11369, 11211, 22650, 11305...",@ LiliyaL62076921 Unique beauty of flowers [UN...,[],[],C8F2526CD1AEA3A8CFC67EB4B79E46D5,E243DF6401359C38C5D21B6FB86A881E,NaN,NaN,NaN,...,0011F45B80474E4FF47250ED24C5482C,27980,25991,False,1493066269,True,NaN,NaN,NaN,1.581058e+09


In [29]:
#d_eng['hashtags_text'].value_counts()

In [30]:
#Replace the hashtags in the text 
valid_pattern = '@ [a-z0-9]+'

#Create a new column to store the list of links
d_eng['mentions_text'] = ""

for index, row in d_eng.iterrows():
    links_list = []
    links_list = re.findall(valid_pattern, tokenizer.decode(d_eng['text_ tokens'][index]))
    for i, s in enumerate(links_list):
        links_list[i] = s.replace(' ','') # I use replace because s.strip() is not working 
        
    d_eng.at[index, 'mentions_text'] = links_list

In [31]:
col = d_eng['mentions_text']
d_eng.drop(labels=['mentions_text'], axis=1, inplace = True)
d_eng.insert(4, 'mentions_text', col)            
            
d_eng.head()

,text_ tokens,text_tokens_no_links,text,hashtags_text,mentions_text,present_links_url,hashtags,tweet_id,present_media,present_links,...,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engaging_user_account_creation,engagee_follows_engager,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp
0,"[16493, 12478, 117, 10105, 42370, 76299, 100, ...","[16493, 12478, 117, 10105, 42370, 76299, 100, ...","Jacob Young, the Syndicate [UNK] s Director of...",[],[],[https://t.co/g5evrQa8va],NaN,39024FBE0136E046D1357196BAECFCA6,GIF,NaN,...,000006829BEADA9EEA695CF0C334B426,2,29,False,1568107028,False,NaN,NaN,NaN,NaN
1,"[14924, 16118, 10114, 12888, 15127, 31204, 101...","[14924, 16118, 10114, 12888, 15127, 31204, 101...",Final week to see my exhibition in the Shircli...,"[#vincennesuniversity, #shircliffgallery, #ins...",[],[https://t.co/PxG5XzTruM],3653868A966576CF17D6A9064889BCED\t7A6710E791A1...,AB3ADBBD011F88D10FE7F6C5FDAB214C,Photo,NaN,...,000269A50293CC2B1E9C72B287FA624E,515,424,False,1254588643,True,NaN,NaN,NaN,1.581090e+09
2,"[138, 19826, 10108, 10105, 10635, 10105, 12250...","[138, 19826, 10108, 10105, 10635, 10105, 12250...",A lot of the time the side that wins is simply...,[],[],[],NaN,675D7920EA2FB4869BA767F5122FB115,NaN,NaN,...,0004C10C480EF88F3EED5822149DC93F,4,125,False,1547955941,False,NaN,NaN,NaN,1.581135e+09
3,"[56898, 137, 42374, 36630, 10797, 90861, 131, ...","[56898, 137, 42374, 36630, 10797, 90861, 131, ...","RT @ readyletsgo27 : Schiff, Nadler and Pelosi...",[],[@readyletsgo27],[],NaN,7EAAE5C243D675AD4CFC2C052B0E0BB5,NaN,NaN,...,000830B836532B8FC5564949E94FEDD6,33439,35598,False,1261862258,True,NaN,1.581012e+09,NaN,NaN
4,"[137, 87043, 10679, 11369, 11211, 22650, 11305...","[137, 87043, 10679, 11369, 11211, 22650, 11305...",@ LiliyaL62076921 Unique beauty of flowers [UN...,[],[],[],C8F2526CD1AEA3A8CFC67EB4B79E46D5,E243DF6401359C38C5D21B6FB86A881E,NaN,NaN,...,0011F45B80474E4FF47250ED24C5482C,27980,25991,False,1493066269,True,NaN,NaN,NaN,1.581058e+09


In [32]:
#d_eng['mentions_text'].value_counts()[:20]

Comparació de twits de diferents tipus: **TopLevel**, **Retweet** i **Quote**

In [33]:
tweet_type = 'TopLevel'
d_eng_toplevel = d_eng.loc[d_eng['tweet_type'] == tweet_type]
engaging_user_id = d_eng_toplevel[engagement_user_type].value_counts().index[0]
d_eng_user = d_eng_toplevel.loc[d_eng[engagement_user_type] == engaging_user_id]

for n in range (20): print(d_eng_user['text'].iloc[n])

Why didn [UNK] t John Bolton complain about this [UNK] nonsense [UNK] a long time ago, when he was very publicly terminated. He said, not that it matters, NOTHING!
[UNK] I like President Trump [UNK] s Tweets ( Social Media ), I like everything about him... and this Ukraine stuff, the trial, the impeachment, this isn [UNK] t t about Ukraine. Donald Trump has committed the two unpardonable sins in the eyes of the Democrats. He beat Hillary Clinton in 2016, and..
Are you better off now than you were three years ago? Almost everyone say YES!
Reports are that basketball great Kobe Bryant and three others have been killed in a helicopter crash in California. That is terrible news!
The Democrat controlled House never even asked John Bolton to testify. It is up to them, not up to the Senate!
..... So, what the hell has happened to @ FoxNews. Only I know! Chris Wallace and others should be on Fake News CNN or MSDNC. How [UNK] s Shep Smith doing? Watch, this will be the beginning of the end for 

In [34]:
tweet_type = 'Retweet'
d_eng_retweet = d_eng.loc[d_eng['tweet_type'] == tweet_type]
engaging_user_id = d_eng_retweet[engagement_user_type].value_counts().index[0]
d_eng_user = d_eng_retweet.loc[d_eng[engagement_user_type] == engaging_user_id]

for n in range (20): print(d_eng_user['text'].iloc[n])

RT @ brithume : McConnell had a choice, to be liked by Democrats and the media or to be effective. He chose effective. By @ wjmcgurn https : [UNK]
RT @ JennaEllisEsq : After everything @ realDonaldTrump has done for this country ( at great personal cost! ) it is so sad that so many are will [UNK]
RT @ SenThomTillis : The only thing we [UNK] ve seen from the # impeachment trial is a political exercise by Chuck and Nancy to prevent us from cont [UNK]
RT @ WhiteHouse : " I am concerned about lowering impeachment standards to fit a paucity of evidence and an abundance of anger. " [UNK] Jonathan Tu [UNK]
RT @ Jim _ Jordan : 2 hours of truth beats over 20 hours of presumptions, assumptions, and hearsay.
RT @ realDonaldTrump : All Democrats should watch this! https://t.co/WFK33pR0Lv
RT @ SteveScalise : President Trump's legal team completely destroyed 3 days of Adam Schiff's false allegations in just 2 hours. Today was de [UNK]
RT @ SenateGOP : Chairman @ SenJohnBarrasso :.. The American peop

In [35]:
tweet_type = 'Quote'
d_eng_quote = d_eng.loc[d_eng['tweet_type'] == tweet_type]
engaging_user_id = d_eng_quote[engagement_user_type].value_counts().index[0]
d_eng_user = d_eng_quote.loc[d_eng[engagement_user_type] == engaging_user_id]

for n in range (20): print(d_eng_user['text'].iloc[n])

Why didn [UNK] t Schiff release this?
True!
Don Lemon, the dumbest man on television ( with terrible ratings! ).
Incredible numbers!
Incredible numbers!
Nadler ripped final argument away from Schiff, thinks Shifty did a terrible job. They are fighting big time!
Thank you!
Charlie, I always knew you are brilliant!
Thank you Roger, it is a big Hoax!
Such a great family!
Don Lemon, the dumbest man on television ( with terrible ratings! ).
Democrats admitting the Impeachment Hoax!
Thank you to Chuck & amp ; all. USMCA is going to be GREAT!
True!
They will never be satisfied!
A very good question!
Congrats to Christine!
Thank you to Chuck & amp ; all. USMCA is going to be GREAT!
They will never be satisfied!
Incredible numbers!


In [36]:
d_eng['present_media'].fillna("No Media", inplace=True)

#Checking tweets with more than one link
for i in range(0,100000):
    if len(d_eng['present_links_url'].iloc[i]) > 1:
        if d_eng['present_media'].iloc[i]:
            print(d_eng['present_media'].iloc[i] + '\n')
        print(d_eng['text'].iloc[i] + '\n')
        

Video

[UNK] This is a very fast moving, constantly changing situation, [UNK] HHS Sec. Azar says about the # coronavirusoutbreak. He says the HHS is working to understand the speed of spread and the severity of the virus. https://t.co/qZuyJkklqg https://t.co/4hiiozzgFH

Video

A Morning in Paris... ♥. https://t.co/jaJduGn5FQ https://t.co/g3lsTIL7Xv

Photo	Photo	Photo	Photo

NEW! We've teamed up with @ STSPhonoCo to bring an exclusive color variant of Castlevania : The Adventure ReBirth to the Yeteemart! It's got killer tracks from Castlevania, Castlevania III, Dracula X, and more, and features art by @ Dreweyes! https://t.co/hoflXYORIv https://t.co/yuBRQhh4D9

Video

From breathtaking beaches to friendly locals, there's so much to love about the DR! @ DarlingEscapes is sharing about her beach vacay, plus the best cities to visit in the Dominican Republic : https://t.co/HU7XgZO9Of https://t.co/XeynSRY70w

Photo

RT @ soompi : LISTEN : # LOONA Previews Mini Album " # " With Highlight Med

Photo

Makes me mad everytime it happens https://t.co/0jmwiRPdnI https://t.co/WPNHP7YXFx

Photo

Day 2 of the # CDL2020 Launch Weekend begins at 2 : 30PM ET!.. https://t.co/gQ6StOebMj https://t.co/gnMRcXiafq

Video

These veggie nuggets are bite - sized goodness!. Shop this recipe in our Tasty iOS app :. https://t.co/MwwxdLLBA7 https://t.co/w6jLlyvhzg

Photo

Botswana registered its first case of the coronavirus on Friday at Sir Seretse Khama International Airport in Gaborone. https://t.co/4TQbTGUAjq https://t.co/so8RGOC7qc

Photo	Photo	Photo	Photo

I am an Asian - American female cinematographer and director currently in Tokyo filming a short romance about two women who meet and get to know each other through art. If you [UNK] re interested, follow the film here :.. https://t.co/TJPSsJeNYQ.. Below are some location scout photos by me. https://t.co/VcQmTZCRXK

Photo

[UNK].. Up next, Tanzania : https://t.co/sPxI9biKcG # PL https://t.co/fm1QoaNmpf

Photo

Construction Workers Inspire Th


The Trump administration has announced an expansion of the travel ban [UNK] one of President Trump's signature policies, which has been derided by critics as an attempt to ban Muslims from the US [UNK] to include six new countries https://t.co/opxAwhlH79 https://t.co/qKEq711Hi3

GIF

I love getting visitors [UNK] today I had 1000 at my FanCentro profile https://t.co/4OB6iCriZL! https://t.co/qaHjLr0w7f

Photo

RT @ ericuman : Jesus... https://t.co/sknaMj9dzn https://t.co/J4H63tNY94

Photo

Just a heads up that tonight's live show is totally interactive... Sure, you can live stream it [UNK] but it'll be way more fun if you participate. Best way is to get our app :.. [UNK] iOS app's here : https://t.co/FaBQdkApMa. [UNK] Android app's here : https://t.co/W5JZHM389V.. # SundanceLiveLabs https://t.co/3V3TbhMS97

Video

" Let's get to work and change the world! ".. # ClimateAction supporter @ BillNye visited UNHQ this week... Here's how you too can help preserve our planet : https://t.co/3JU


# jazz # art " Every child is an artist. The problem is how to remain an artist once we grow up. " - Pablo Picasso. • https://t.co/yx5yPGZ7sQ • https://t.co/WYul91lbqr

GIF

DREAM SKETCH : Page # 4. # 재민 # jaemin # nctdream. https://t.co/42lKCz079c https://t.co/uWaJANXLRh

Photo

Minnesota took the @ NCAAsoftball postseason by storm in 2019, earning a trip to the # WCWS. With NFCA first - team All - American @ amber _ fiser back in the circle, @ GopherSoftball looks to be a strong contender for 2020. See them at the # NFCAD1Leadoff in # 10Days. https://t.co/NJNkkMgvVf https://t.co/nF5pND7oAv

Photo

Stars Shine On The Red Carpet Of 29th Seoul Music Awards https://t.co/rexrVIBPX2 https://t.co/XFJdAsqd2z

Photo	Photo

[UNK] but make it Fashion [UNK]. Some blue on your timeline [UNK].. https://t.co/zFUXOGwhg3 https://t.co/hQ61zAJdQy

Photo

Live now on BBC Parliament : the Trump impeachment trial continues in the US Senate... Watch here : https://t.co/JPgnCAgJ1K https://t.co/M2CupFAzai




Middle East # PeacePlan is the toughest deal, but we have an obligation to the world - Trump.. https://t.co/ryLpOsmCkD https://t.co/LIK8A8lqO6

Photo

RT @ FarmsNews : The president USMCA a [UNK] tremendous breakthrough [UNK] for U. S. farmers : https://t.co/bK7HRQOv7A https://t.co/1FALBDn2cB

Photo

Taking inspiration from some of the most iconic Jordans of all time, King James gets the " Infrared " treatment with this great new colourway of the Nike LeBron 17 that is now up on Nike CA for $ 165 + free shipping! https://t.co/A4KAV3GoUa https://t.co/iW0gPtOSSp

Video

Is the posterboy of quirky cinema, @ ayushmannk, most sensible actor in Bollywood?. Watch # InDaClub with @ Sushant _ IToday. More videos at https://t.co/wMGGKJPL5n https://t.co/yQXPeZrpXN

Photo	Photo	Photo	Photo

The Temple Zone. The third story from magazine'Cruella 1 '. https://t.co/t0FCGnwdN5.. @ BritLeatherGirl. @ CruellaDarkStar. @ EmpireBlg https://t.co/rQGl4gZ81b

Photo

Speaking of honor... After repping the [U

...


B1 AND / OR B2 ENGINEERS - FALCON, EMBRAER, GULFSTREAM.. We're recruiting B1 and / or B2 Engineers with # Gulfstream # Bombardier # Falcon type ratings on behalf of our leading business aviation client in Switzerland... Permanent roles, paying up to 80, 000 CHF... https://t.co/KIlF9RdzUq https://t.co/p8jw7QN2G5

Photo

Grab your favorite floaties. Stream new episodes free only on The CW App : https://t.co/fCp2T1NqrL # Legacies https://t.co/KWKOr9iain

Photo

When her service dog Hayley needed a new cape, Brittany had one custom - embroidered to feature Ellie's tattoo from The Last of Us Part II. Thank you for sharing this with us! It made our day... Submit your own creations here for a chance to be featured : https://t.co/rkNPoi2lDv https://t.co/JaHShC94XU

Photo

Justin Verlander Had The Balls To Joke About The Cheating Astros During The BBWAA Dinner Last Night And CC Sabathia Was Furious. https://t.co/YZJs6CVkss https://t.co/8xSCAxaurF

Photo

The NBA is reformatting the All - Star 


They made this boi too pure and innocent. Here's the full update translation : https://t.co/XaCGNbOPdL https://t.co/vqG5LvC4x0

Photo

New law soon for permission to set up industries in 7 days in MP : Kamal Nath.. https://t.co/wYQFweW20W https://t.co/Y5sst8zZXD

No Media

RT @ risingstarpr : Boobjob Blogger @ SofiRyanxoxo Stars in [UNK] Look at Her Now [UNK] Scene! https://t.co/xSlGDgecdS [UNK] # newrelease https://t.co/BKgNXdD [UNK]

Photo

Go Fuck Yourself, a song by Two Feet on Spotify. https://t.co/cR3sVa5ZDp https://t.co/2Dse6oLKmA

Photo	Photo

Hey # RVA folks, if you haven't made plans for tonight, join us, @ thcjusticenow, @ RISEforYouth, Del. @ JoshuaCole and many others at @ Hardywood to talk about marijuana reform at the General Assembly this session... We'll have swag & amp ; resources for everyone! https://t.co/xv1mXTq0of https://t.co/VdIiYuJTh7

Photo

11 Indian fishermen held off Sri Lankan coast for fishing in territorial waters.. READ : https://t.co/AefxAiDjsg https:


I want to fuck your ass, but I am not just going to slam my strap in there. New clip : I [UNK] m Gonna Take My Time # pegging https://t.co/QwGOhDaJXQ https://t.co/8At1mkbXWp

Photo

You're about to unleash SHOOKY's special...! [UNK].. BT21 UNIVERSE ANIMATION EP07 Part 1,. COMING SOON. & gt ; https://t.co/nQw5KOZWwL.. # January29th # Part1 # January31st # Part2 # StayTuned # BT21 _ UNIVERSE # ANIMATION # EP07 # SHOOKY # ChildhoodMemories # CRUNCHYSQUAD # BT21 https://t.co/lTkgTK6Vvr

Photo	Photo	Photo	Photo

I always wanted George to call out Fake Lionel. You weren [UNK] t fooling anybody, Fake Lionel!.. [UNK] https://t.co/2MP13xmYZQ https://t.co/lUszJuHTND

Video

The apogee of handmade, the # DiorCouture Spring - Summer 2020 collection by Maria Grazia Chiuri saw the use of a very special pleating technique. Discover more about it here and https://t.co/FGm6d8qua9.. # DiorSavoirFaire https://t.co/UmApRRtcr9

No Media

Tonight is our @ iHeartRadio LIVE show presented by HOT TOPIC! Watch

Photo	Photo

What are you waiting for? [UNK].. These and so much more all waiting for you : https://t.co/gYNRSdSGo3 https://t.co/FpjTnL0gkf

Video

Hat - trick!.. Stormy Ireland is in fine fettle and beats the boys down in trip in the Limestone Lad for @ PTownend and Willie Mullins @ NaasRacecourse.. Results [UNK] https://t.co/glajGV55GR https://t.co/0c6LUINElM

Photo

I can't believe my Auntie Bobrisky blocked me - Effeminate dancer, James Brown cries out https://t.co/bR5Lbb3Okh https://t.co/b9U9MB2AGe

No Media

RT @ DanWhitCongress : Don [UNK] t forget to check out my website.. https://t.co/3ntAbAnycD.. and my Facebook.. https://t.co/2JKac5I2Mv

No Media

RT @ Variety : Popstar @ NikitaDragun turning heads with this futuristic, pearly white gown # GRAMMYs https://t.co/rU1O1bxnQm https://t.co/0OKl [UNK]

Photo

Judicial Watch is in federal court trying to uncover the controversial secret subpoenas issued by Rep. Schiff & amp ; the House Intel Committee for phone records, including th


It's time to party! And with [UNK] Kanda Kanda [UNK] on the speakers, we'll be dancing all'Raat.'Listen to all the hits on the playlist [UNK] Electro Mix, [UNK] here on Gaana : https://t.co/OIUkH47W0i https://t.co/9X9LMQblGR

Photo

Nightwing Has Come to # Batman Beyond - And He's Looking for [ SPOILER ]. https://t.co/uzTE5KwJtC https://t.co/Je3ibLLxuO

Video

. @ ianbremmer and I dive deep into the biggest global threats in 2020 [UNK] chief among them being US politics and the upcoming election : https://t.co/af3fhPQQ40 https://t.co/9xlXzcPeoN

Video

RT @ AhegaoOnline : Nee, [UNK] Shiyo. full hentai - https://t.co/VQM1ezs3g4.. # hentai # hentaivideo # ahegao https://t.co/daWQMikLUp

GIF

Gonna hop on https://t.co/f6UA5bNuqZ in about 30 minutes or so to rock out the remaster of Warcraft III. Let's travel back in time to a world where everyones voice was different, lore was way less confusing, and Mojomaker was a just a twinkle in his daddy's eye. https://t.co/FwbtXWIqsY

Photo

RT @ 

In [37]:
#Checking links with No Media
for i in range(0,100000):
    if len(d_eng['present_links_url'].iloc[i]) > 1:
        if d_eng['present_media'].iloc[i] == 'No Media':
            print(d_eng['present_media'].iloc[i] + '\n')
            print(d_eng['text'].iloc[i] + '\n')

No Media

RT @ reflectivelatex : Tattoos, transparent latex and vacbeds. Oh my! With @ thecamdamage. Full video :.. https://t.co/xABJG6K1OD https://t.co/j [UNK]

No Media

[UNK] By Popular [UNK] This Hoodie Sweater Dress is so Bomb!. [UNK] Now ~ Pay [UNK]. https://t.co/z4ezXTSPRB......... [UNK] Online and Schedule Your Houston Showroom Pick Up........... # dresses # hoodiedresses [UNK] https://t.co/dPPsizsA8V

No Media

Tonight is our @ iHeartRadio LIVE show presented by HOT TOPIC! Watch on @ thecw app or on https://t.co/PsYe4Fqk9f at 9pm ET / 6pm PT! # iHeartBTS. https://t.co/gWsJqt4GxC

No Media

RT @ itskarinareyes : My shop has been updated! Go check it out, I have a lot of cute things on https://t.co/elYAuf4uT8 [UNK] https://t.co/BLbRRb [UNK]

No Media

RT @ wizkhalifa : If you party with me you party with @ McQueenVF @ TylaYaweh. # HighRightNowRemix. [UNK] : https://t.co/jKD4Qz3MxX https://t.co/NYuOb [UNK]

No Media

The Scenic Route to York feat : Night Heron Soundtrack https://


[UNK] Show. [UNK] January. [UNK] 1 : 00 am. [UNK] E! https://t.co/MCFIf2VEhw,. CBS YT ( not confirmed ) https://t.co/h6LqHoeE9j,. https://t.co/ilbonOGyJK +

No Media

New post ( Baby Sitter Fucked ) has been published on https://t.co/zsAVApjgQg XXX Sex Videos and Porn Star Movies - https://t.co/TXaukBZx0B

No Media

RT @ RecordingAcad : @ bts _ bighit is back. [UNK] # BTSArmy, we'll see you tonight. https://t.co/Q55i1gEyma # UnexpectEverything https://t.co/xbKkBxP [UNK]

No Media

RT @ OANN : # BREAKING - Report : Kobe Bryant And Four Others Killed In Calif. Helicopter Crash https://t.co/ebahgkny5q # OANN https://t.co/PQ2f [UNK]

No Media

Your key source for ACPS # specialeducation resources? The ACPS Parent Resource Center! Follow them :. Twitter @ LearnwithThePRC. Facebook : https://t.co/BlEymXZlkc. Newsletter : https://t.co/nYm0o6oiuF

No Media

RT @ RockyVallarta : My arab friend stopped by my place to face fuck me real quick. Coming son on :.. https://t.co/Qn2GrFdgDC. https://t.

No Media

RT @ kwillard99 : Willard named District Coach of the Year | Butlerbulletin | https://t.co/X1h4U7qQrL https://t.co/uRlhaId9It

No Media

Mac users can now download Microsoft [UNK] s new Surface Duo SDK https://t.co/fndquRbQIp https://t.co/MqFRcwdGzf

No Media

The Times and Sunday Times to launch digital ad - free talk radio station in partnership with radio group Wireless, which is also owned by News UK ( @ charlottetobitt / Press Gazette ).. https://t.co/2yNwRO28eJ. https://t.co/FHeub5bZQr

No Media

RT @ iHeartTheaterLA : Truly the luckiest fan ever! [UNK].. Watch the show live on @ TheCW app or https://t.co/DG3s8hqcFv : https://t.co/TuRzYsSpfN [UNK]

No Media

[UNK] You don [UNK] t lead by pointing and telling people some place to go. You lead by going to that place and making a case... For more Sales & amp ; Marketing insights visit https://t.co/kqgUkwy7wD.. [UNK] Read the full article here.... https://t.co/gTj2mfi8df

No Media

FIRST STREAM IS ON!!!.. https://t.co/7BBj2

No Media

RT @ staronline : Thai GDP growth lags behind expectations this year and last. https://t.co/eUgXT6Vzj1 https://t.co/eUgXT6Vzj1

No Media

RT @ kemaco _ : Watch / Download full video on MMM100 : https://t.co/vlYTIvTHMF. # # # # # # # # # # # # #. Porn Casting : https://t.co/wQsQTdeFbs. # # # # # # # [UNK]

No Media

Our goal is to raise $ 500 and all amounts are appreciated as every little bit helps!.. Twitch : https://t.co/ShwDSaWmYS. Where to donate : https://t.co/1QBHSwmuiS.. # remnantgamers # gaming # podcasters # AustralianRedCross # Australiafires # climatechange # remnantgamersgiveback # charity

No Media

BBC News : How John Major complained about broadcaster to US President | UK | News | https://t.co/tZJVkFT2sW https://t.co/yQiND5GDxf

No Media

anti # 2A gun grabbers want to flip Texas blue like they did to Virginia.. [UNK] 28 Jan 2020.. [UNK] Gary Gates @ GatesforTexas. Texas House District 28 # HD28 Fort Bend. [UNK] : / / t. co / J7FCHpbMx0.. [UNK] Info : https://t


You can find Tony Basilio's Next Level Network Family of Podcasts on :. - TuneIn : https://t.co/rBXVgCPuqx. - Stitcher : https://t.co/SwE319WuQ6. - Overcast : https://t.co/2W1gK3iVB1... - Google : https://t.co/OtECwU6F9p. - iHeartRadio : https://t.co/gQeFaaHcyZ

No Media

Playboy Cybergirl - Emy Richardson! Best of # Playboy # Erotic! See more & gt ; & gt ; https://t.co/WqrAmpzjBt & lt ; & lt ; https://t.co/hNPrqcR3br

No Media

RT @ AlexDicksmen : @ realhellaunik OF : https://t.co/kAOHvws2ps.. PH : https://t.co/SniPc82M47.. MV : https://t.co/MXcXQVJoW5.. J4F : https : / / t. co [UNK]

No Media

RT @ TheKhiLavene1 : When 2 tops collide.. We call in a 3rd for help.. Part 1 [UNK] feat @ KingFame _ 89 https://t.co/wBtZvpV2Vq https://t.co/UYC8Oh [UNK]

No Media

RT @ PhilstarShowbiz : WATCH : Catriona Gray willing to be Gerald Anderson's next leading lady | https://t.co/cDVNXRH3tw https://t.co/Vdu4N6yr [UNK]

No Media

A bestseller 24 years ago. A case study on how ordinary people can be 

In [43]:
d_eng['link_to_media'] = ""

for index, row in d_eng.iterrows():
    if row['present_links_url'] != []:
        d_eng.at[index, 'link_to_media'] = row['present_links_url'][-1]
d_eng.head()

,text_ tokens,text_tokens_no_links,text,hashtags_text,mentions_text,present_links_url,hashtags,tweet_id,present_media,present_links,...,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engaging_user_account_creation,engagee_follows_engager,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp,link_to_media
0,"[16493, 12478, 117, 10105, 42370, 76299, 100, ...","[16493, 12478, 117, 10105, 42370, 76299, 100, ...","Jacob Young, the Syndicate [UNK] s Director of...",[],[],[https://t.co/g5evrQa8va],NaN,39024FBE0136E046D1357196BAECFCA6,GIF,NaN,...,2,29,False,1568107028,False,NaN,NaN,NaN,NaN,https://t.co/g5evrQa8va
1,"[14924, 16118, 10114, 12888, 15127, 31204, 101...","[14924, 16118, 10114, 12888, 15127, 31204, 101...",Final week to see my exhibition in the Shircli...,"[#vincennesuniversity, #shircliffgallery, #ins...",[],[https://t.co/PxG5XzTruM],3653868A966576CF17D6A9064889BCED\t7A6710E791A1...,AB3ADBBD011F88D10FE7F6C5FDAB214C,Photo,NaN,...,515,424,False,1254588643,True,NaN,NaN,NaN,1.581090e+09,https://t.co/PxG5XzTruM
2,"[138, 19826, 10108, 10105, 10635, 10105, 12250...","[138, 19826, 10108, 10105, 10635, 10105, 12250...",A lot of the time the side that wins is simply...,[],[],[],NaN,675D7920EA2FB4869BA767F5122FB115,No Media,NaN,...,4,125,False,1547955941,False,NaN,NaN,NaN,1.581135e+09,
3,"[56898, 137, 42374, 36630, 10797, 90861, 131, ...","[56898, 137, 42374, 36630, 10797, 90861, 131, ...","RT @ readyletsgo27 : Schiff, Nadler and Pelosi...",[],[@readyletsgo27],[],NaN,7EAAE5C243D675AD4CFC2C052B0E0BB5,No Media,NaN,...,33439,35598,False,1261862258,True,NaN,1.581012e+09,NaN,NaN,
4,"[137, 87043, 10679, 11369, 11211, 22650, 11305...","[137, 87043, 10679, 11369, 11211, 22650, 11305...",@ LiliyaL62076921 Unique beauty of flowers [UN...,[],[],[],C8F2526CD1AEA3A8CFC67EB4B79E46D5,E243DF6401359C38C5D21B6FB86A881E,No Media,NaN,...,27980,25991,False,1493066269,True,NaN,NaN,NaN,1.581058e+09,
